<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/nlp/%EB%AC%B8%EC%9E%A5%EC%9D%BC%EA%B4%80%EC%84%B1(%EC%9D%91%EC%A7%91%EC%84%B1)_%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84%EA%B8%B0%EB%B0%98_%EB%89%B4%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install sentence_transformers datasets

     |████████████████████████████████| 78 kB 5.4 MB/s 
     |████████████████████████████████| 290 kB 27.2 MB/s 
     |████████████████████████████████| 2.9 MB 36.6 MB/s 
     |████████████████████████████████| 3.3 MB 33.9 MB/s 
     |████████████████████████████████| 1.2 MB 21.3 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 45.0 MB/s 
     |████████████████████████████████| 596 kB 40.2 MB/s 
     |████████████████████████████████| 1.3 MB 41.4 MB/s 
     |████████████████████████████████| 125 kB 58.2 MB/s 
     |████████████████████████████████| 243 kB 51.4 MB/s 
     |████████████████████████████████| 160 kB 48.2 MB/s 
     |████████████████████████████████| 271 kB 53.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=5d491d490bf3cac50c814e0a77c81a8365d4d383ed5a01207ba98bb4e8a9700a
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da700

In [4]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [5]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [6]:
model_name = "klue/roberta-base"
train_batch_size = 32
num_epochs = 4
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [7]:
embedding_model = models.Transformer(model_name)

2021-10-28 08:08:51 - Attempting to acquire lock 140651325971664 on /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a.lock
2021-10-28 08:08:51 - Lock 140651325971664 acquired on /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a.lock


Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

2021-10-28 08:08:52 - Attempting to release lock 140651325971664 on /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a.lock
2021-10-28 08:08:52 - Lock 140651325971664 released on /root/.cache/huggingface/transformers/a96469ca2a238496d435a0e9e202f261119c146a0326444b6d68ae1adc35e04f.85b0b02ba2a483f3adb8a60ab70dbd875768fcd5e6cdb21a593c6e02fdffac3a.lock
2021-10-28 08:08:52 - Attempting to acquire lock 140651320187728 on /root/.cache/huggingface/transformers/b204e0dc0a3b8fd45b35e7fcefd97c5f839b86c14aea510f1eb38fb8469e23d8.57d3cd0dfa80e5a249a776870dc87b6da993900685a271086750174009115320.lock
2021-10-28 08:08:52 - Lock 140651320187728 acquired on /root/.cache/huggingface/transformers/b204e0dc0a3b8fd45b35e7fcefd97c5f839b86c14aea510f1eb38fb8469e23d8.57d3cd0dfa80e5a249a776870dc87b6da993900685a271086750174009115320.lock


Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

2021-10-28 08:09:07 - Attempting to release lock 140651320187728 on /root/.cache/huggingface/transformers/b204e0dc0a3b8fd45b35e7fcefd97c5f839b86c14aea510f1eb38fb8469e23d8.57d3cd0dfa80e5a249a776870dc87b6da993900685a271086750174009115320.lock
2021-10-28 08:09:07 - Lock 140651320187728 released on /root/.cache/huggingface/transformers/b204e0dc0a3b8fd45b35e7fcefd97c5f839b86c14aea510f1eb38fb8469e23d8.57d3cd0dfa80e5a249a776870dc87b6da993900685a271086750174009115320.lock


...

2021-10-28 08:09:10 - Attempting to release lock 140651325402512 on /root/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a.lock
2021-10-28 08:09:10 - Lock 140651325402512 released on /root/.cache/huggingface/transformers/e8441a174492958462b6b16b6db8f1e7253cd149ca779522cadd812d55091b89.d1b86bed49516351c7bb29b19d7e7be2ab53b931bcb1f9b2aacfb71f2124d25a.lock
2021-10-28 08:09:11 - Attempting to acquire lock 140651319832016 on /root/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2.lock
2021-10-28 08:09:11 - Lock 140651319832016 acquired on /root/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2.lock


Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

2021-10-28 08:09:12 - Attempting to release lock 140651319832016 on /root/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2.lock
2021-10-28 08:09:12 - Lock 140651319832016 released on /root/.cache/huggingface/transformers/233a5b2c17873a8477b62dd92a02092a9937759e924a5f22b111becebb8aba5e.44c30ade4958fcfd446e66025e10a5b380cdd0bbe9b3fb7a794f357e7f0f34c2.lock
2021-10-28 08:09:13 - Attempting to acquire lock 140651318897488 on /root/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23.lock
2021-10-28 08:09:13 - Lock 140651318897488 acquired on /root/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23.lock


Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

2021-10-28 08:09:13 - Attempting to release lock 140651318897488 on /root/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23.lock
2021-10-28 08:09:13 - Lock 140651318897488 released on /root/.cache/huggingface/transformers/9d0c87e44b00acfbfbae931b2e4068eb6311a0c3e71e23e5400bdf57cab4bfbf.70c17d6e4d492c8f24f5bb97ab56c7f272e947112c6faf9dd846da42ba13eb23.lock


In [8]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [9]:
model = SentenceTransformer(modules=[embedding_model, pooler])

2021-10-28 08:09:14 - Use pytorch device: cuda


In [10]:
datasets = load_dataset("klue", "sts")

Downloading:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
datasets["train"][0]

{'guid': 'klue-sts-v1_train_00000',
 'labels': {'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714},
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'source': 'airbnb-rtt'}

In [12]:
testsets = load_dataset("kor_nlu", "sts")

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/89.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/66.1k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset kor_nlu downloaded and prepared to /root/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
train_samples = []
dev_samples = []
test_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)

In [14]:
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)

In [15]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

In [16]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

2021-10-28 08:09:35 - Warmup-steps: 146


In [17]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-28 08:16:05 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2021-10-28 08:16:09 - Cosine-Similarity :	Pearson: 0.8773	Spearman: 0.8739
2021-10-28 08:16:09 - Manhattan-Distance:	Pearson: 0.8776	Spearman: 0.8725
2021-10-28 08:16:09 - Euclidean-Distance:	Pearson: 0.8778	Spearman: 0.8725
2021-10-28 08:16:09 - Dot-Product-Similarity:	Pearson: 0.8706	Spearman: 0.8646
2021-10-28 08:16:09 - Save model to output/training_klue_sts_klue-roberta-base-2021-10-28_08-08-51


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-28 08:22:17 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2021-10-28 08:22:21 - Cosine-Similarity :	Pearson: 0.8870	Spearman: 0.8859
2021-10-28 08:22:21 - Manhattan-Distance:	Pearson: 0.8869	Spearman: 0.8825
2021-10-28 08:22:21 - Euclidean-Distance:	Pearson: 0.8876	Spearman: 0.8834
2021-10-28 08:22:21 - Dot-Product-Similarity:	Pearson: 0.8755	Spearman: 0.8714
2021-10-28 08:22:21 - Save model to output/training_klue_sts_klue-roberta-base-2021-10-28_08-08-51


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-28 08:28:30 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2021-10-28 08:28:34 - Cosine-Similarity :	Pearson: 0.8906	Spearman: 0.8902
2021-10-28 08:28:34 - Manhattan-Distance:	Pearson: 0.8898	Spearman: 0.8857
2021-10-28 08:28:34 - Euclidean-Distance:	Pearson: 0.8910	Spearman: 0.8865
2021-10-28 08:28:34 - Dot-Product-Similarity:	Pearson: 0.8812	Spearman: 0.8763
2021-10-28 08:28:34 - Save model to output/training_klue_sts_klue-roberta-base-2021-10-28_08-08-51


Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2021-10-28 08:34:44 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2021-10-28 08:34:48 - Cosine-Similarity :	Pearson: 0.8922	Spearman: 0.8920
2021-10-28 08:34:48 - Manhattan-Distance:	Pearson: 0.8913	Spearman: 0.8870
2021-10-28 08:34:48 - Euclidean-Distance:	Pearson: 0.8922	Spearman: 0.8880
2021-10-28 08:34:48 - Dot-Product-Similarity:	Pearson: 0.8820	Spearman: 0.8776
2021-10-28 08:34:48 - Save model to output/training_klue_sts_klue-roberta-base-2021-10-28_08-08-51


In [18]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

apt install curl
apt install git
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
Done
Successfully Installed
Now you can use Mecab
from konlpy.tag import Mecab
mecab = Mecab()
사용자 사전 추가 방법 : https://bit.ly/3k0ZH53
NameError: name 'Tagger' is not defined 오류 발생 시 런타임을 재실행 해주세요
블로그에 해결 방법을 남겨주신 tana님 감사합니다.


In [19]:
from konlpy.tag import *
okt = Okt()

In [81]:
import pandas as pd

blog_data = pd.read_csv('/content/drive/MyDrive/nlp/kor_news.csv', encoding='utf-8')

In [87]:
contents = blog_data[10:50]
contents.reset_index(inplace=True)

In [88]:
contents.head()

,index,id,contents,category
0,10,NIRW2000000001.11,"상처받은 도시인 치유하는 그곳, 환자 아닌 고객 대접 암스테르담에서 남동쪽을 향해...",생활
1,11,NIRW2000000001.12,다시 꿈틀대는 남북 경협주 '개미무덤 주의보' 북미가 2차 정상회담을 오는 2월말...,사회
2,12,NIRW2000000001.13,"김수현 정책실장 ""부동산 조금이라도 불안하면 지체없이 추가대책"" 청와대 김수현 정...",경제
3,13,NIRW2000000001.14,"김경수 ""스마트산단 조속 지정해 달라"" 당·정에 경남 유치 요청 김경수 경남지사가...",정치
4,14,NIRW2000000001.15,'조작이 판을 친다'… 반쪽짜리 '자동차 자기인증제' 최근 공정거래위원회가 연비 ...,생활


In [89]:
#이어서 contents 전처리(정제)
data_colname = 'contents'
contents[data_colname] = contents[data_colname].str.replace("\(.*\)|\s-\s.*"," " ,regex=True)
contents[data_colname] = contents[data_colname].str.replace("\[.*\]|\s-\s.*"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("\<.*\>|\s-\s.*"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("무단전재 및 재배포 금지"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("무단 전재 및 재배포 금지"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("©"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("ⓒ"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace("저작권자"," ",regex=True)
contents[data_colname] = contents[data_colname].str.replace(".* 기자", " ", regex=True) #기자 이름에서 오는 유사도 차단
contents[data_colname] = contents[data_colname].str.replace("사진 = .*", " ", regex=True) #사진 첨부 문구 삭제
contents[data_colname] = contents[data_colname].str.replace("사진=.*", " ", regex=True) #사진 첨부 문구 삭제
contents[data_colname] = contents[data_colname].str.replace('\"', "",regex=True)
contents[data_colname] = contents[data_colname].str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
contents[data_colname] = contents[data_colname].str.replace("\n"," ")
contents[data_colname] = contents[data_colname].str.replace("\r"," ")
contents[data_colname] = contents[data_colname].str.replace("\t"," ")
contents[data_colname] = contents[data_colname].str.replace( "\’" , "", regex=True)
contents[data_colname] = contents[data_colname].str.replace("[ ]{2,}"," ",regex=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [90]:
!pip install kss==2.5.1

In [91]:
contents['morphs'] = -1
contents['morphs_arr'] = -1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [92]:
contents.head()

,index,id,contents,category,morphs,morphs_arr
0,10,NIRW2000000001.11,"상처받은 도시인 치유하는 그곳, 환자 아닌 고객 대접 암스테르담에서 남동쪽을 향해...",생활,-1,-1
1,11,NIRW2000000001.12,다시 꿈틀대는 남북 경협주 '개미무덤 주의보' 북미가 2차 정상회담을 오는 2월말...,사회,-1,-1
2,12,NIRW2000000001.13,들과 간담회를 열고 '지난해에 비해 부동산 상승세가 꺾였다는 평가에 대해 어떻게 ...,경제,-1,-1
3,13,NIRW2000000001.14,김경수 스마트산단 조속 지정해 달라 당·정에 경남 유치 요청 김경수 경남지사가 제...,정치,-1,-1
4,14,NIRW2000000001.15,'조작이 판을 친다'… 반쪽짜리 '자동차 자기인증제' 최근 공정거래위원회가 연비 ...,생활,-1,-1


In [93]:
contents['문서 응집도'] = '-'
contents['표준 편차'] = '-'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
from kss import split_sentences

for idx, doc in enumerate(contents['contents']):
  print(idx)
  sentences = split_sentences(doc)
  result_arr = []
  result = []
  for sentence in sentences:
    clean_words = []
    for word in okt.pos(sentence, stem=True):
      if word[1] in ['Noun', 'Verb', 'Adjective']: #명사, 동사, 형용사
        clean_words.append(word[0])
    temp_sentence = ' '.join(clean_words)
    result_arr.append(clean_words)
    result.append(temp_sentence)
  contents['morphs'][idx] = result
  contents['morphs_arr'][idx] = result_arr

In [ ]:
import math

import numpy
doc_all_coherence=[]
for index, doc in enumerate(contents['morphs']):
  all_coherence = []
  if(len(doc)==1):
    continue
  for i, sentence in enumerate(doc):
    if(i==len(doc)-1): 
      break
    query_embedding_a = model.encode(doc[i])
    query_embedding_b = model.encode(doc[i+1])

    # 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
    cos_scores = util.pytorch_cos_sim(query_embedding_a, query_embedding_b)[0]
    all_coherence.append(cos_scores)
  contents['문서 응집도'][index] = str(numpy.mean(all_coherence))
  contents['표준 편차'][index] = str(numpy.std(all_coherence))
  doc_all_coherence.append(numpy.mean(all_coherence))

print(doc_all_coherence)
print('전체 평균 코사인 유사도: ',numpy.mean(doc_all_coherence))
print('표준 편차: ',numpy.std(doc_all_coherence))

In [96]:
contents.to_html('/content/drive/MyDrive/nlp/lucy_news_result.html')

In [98]:
print(doc_all_coherence)
print('전체 평균 코사인 유사도: ',numpy.mean(doc_all_coherence))
print('표준 편차: ',numpy.std(doc_all_coherence))

[0.27466652, 0.40412477, 0.18741666, 0.35626212, 0.32027614, 0.5483499, 0.3702296, 0.4432788, 0.3711118, 0.48933214, 0.5253223, 0.32917202, 0.37019056, 0.17954156, 0.5058343, 0.3790767, 0.29998967, 0.5323354, 0.31553712, 0.28212497, 0.46123612, 0.31094545, 0.4151761, 0.3171022, 0.33540303, 0.4274043, 0.33631185, 0.49427766, 0.2601267, 0.44710764, 0.33610305, 0.40145075, 0.3184734, 0.37625098, 0.28870425, 0.44029844, 0.3044071]
전체 평균 코사인 유사도:  0.37175548
표준 편차:  0.08952196
